In [1]:
from tqdm import tqdm
from PIL import Image
import numpy as np
Image.MAX_IMAGE_PIXELS = None
from torchvision import transforms
import torch
from torch.utils.data import Dataset
import os
from multiprocessing import Pool
# import umap
import numpy as np
import matplotlib.pyplot as plt
import sklearn

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

from sklearn.metrics import f1_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, cohen_kappa_score, accuracy_score
import torch.nn.functional as F
import sys
import time

import shutil
import os

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import cv2
import time
from sklearn.preprocessing import label_binarize

import timm
from metrics import report
from pathlib import Path

import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, Dataset
from tqdm import tqdm
from rl_benchmarks.metrics import *
from pathlib import Path
from rl_benchmarks.trainers.torch_trainer import TorchTrainer
from rl_benchmarks.models.slide_models.meanpool import MeanPool
from rl_benchmarks.models.slide_models.chowder import Chowder
from rl_benchmarks.models.slide_models.dsmil import DSMIL
from rl_benchmarks.models.slide_models.abmil import ABMIL
from rl_benchmarks.models.slide_models.hiptmil import HIPTMIL
from rl_benchmarks.models.slide_models.transmil import TransMIL

import sys
sys.path.append('/home/yuxin/bme/BCaCAD/model')
from patch_based_test.img import QiLuROI

def get_model(model_name, in_dim, out_dim):
    if model_name == 'abmil':
        model = ABMIL(in_dim, out_dim, d_model_attention=128,temperature= 1.0, mlp_hidden= [128, 64])
    elif model_name == 'chowder':
        model = Chowder(in_dim, out_dim, n_top= 2,n_bottom= 2,tiles_mlp_hidden= [128], mlp_hidden= [128, 64])
    elif model_name == 'dsmil':
        model = DSMIL(in_dim, out_dim, d_tiles_values= 32,d_tiles_queries= 32,passing_values= False,tiles_scores_mlp_hidden=[200,100],
                        tiles_queries_mlp_hidden=[200,100], mlp_hidden=[200,100])
    elif model_name == 'hiptmil':
        model = HIPTMIL(in_dim, out_dim)
    elif model_name == 'transmil':
        model = TransMIL(in_dim, out_features=out_dim)
    elif model_name == 'meanpool':
        model = MeanPool(in_dim, out_dim)
    else:
        raise 'model not found'
    return model
input_dim = 768
out_dim = [3,3]
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
model_names = [ 'abmil']
models={}
model_dir = Path('/mnt/hd0/project/bcacad/model/roi_models/model6')
for model_name in model_names:
    model = get_model(model_name, input_dim, out_dim).to(device)
    model.load_state_dict(torch.load(model_dir / f'{model_name}.pth'))
    models[model_name] = model.to(device)

In [3]:
import numpy as np
path = '/mnt/hd0/project/bcacad/model/roi_features/suqh_all_patch/test/dcis-2/01594.17-2-10x-x33298-y39454-w2091-h2988-dcis-2.png.npy'
features = torch.from_numpy(np.load(path)).to(device)
features = features.unsqueeze(0)
features.shape

torch.Size([1, 6, 768])

In [4]:
roi_feats = model.extract_features(features)
roi_feats.shape

torch.Size([1, 128])

In [5]:
roi_model = model

In [6]:
class WSI(Dataset):
    def get_label(self, type):
        D = {
            'normal':[0,0],
            'dcis-1':[1,0],
            'dcis-2':[1,1],
            'dcis-3':[1,2],
            'ibc-1':[2,0],
            'ibc-2':[2,1],
            'ibc-3':[2,2],
        }
        label =  D[type]
        return label[0],label[1]
    
    def __init__(self, feature_dir, phase='test'):
        folder = Path(feature_dir)
        wsi_dirs = list(folder.glob('*/*'))
        # self.labels = [self.get_label(path.parent.name) for path in wsi_dirs]
        self.labels = []
        for path in wsi_dirs:
            label = path.parent.name
            cvtlabel = self.get_label(label)
            self.labels.append(cvtlabel)
        # self.features = [np.load(path) for path in feature_paths]
        self.features = []
        for wsi_dir in wsi_dirs:
            feature_paths = list(wsi_dir.rglob('**/*.npy'))
            features = []
            for path in feature_paths:
                patch_features = torch.from_numpy(np.load(path)).to(device).unsqueeze(0)
                roi_feats = roi_model.extract_features(patch_features)
                features.append(roi_feats)
            self.features.append(features)
        self.wsis = wsi_dirs
        self.phase=phase
        
    def __len__(self):
        return len(self.wsis)
    def __getitem__(self, item):
        features = self.features[item]
        # wsi = self.wsis[item]
        if self.phase == 'train':
            indices = np.random.choice(len(features), 6, replace=True)
            features = torch.cat([features[i] for i in indices], dim=0)
            return features, self.labels[item]
        features = torch.cat(features, dim=0)
        return features, self.labels[item]
    
    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images,  labels = tuple(zip(*batch))

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        # masks = torch.as_tensor(masks)

        return images, labels

In [13]:
cfg = dict(
    epoch=200,
    bs=8,
    # train_tiles = 4,
)

In [8]:
criterion = nn.CrossEntropyLoss()
merics = {'acc': compute_multiclass_accuracy, 'auc': compute_mean_one_vs_all_auc}
input_dim = 128
out_dim = [3,3]
tasks = ['type', 'nonibc', 'ibc']
class_names = {
    'type': ['Normal', 'nonIBC', 'IBC'],
    'nonibc': ['Low', 'Medium', 'High'],
    'ibc': ['Low', 'Medium', 'High'],
}

In [9]:
feature_root = Path('/mnt/hd0/project/bcacad/model/wsi_features')
train_set = WSI(feature_root / 'suqh_all_patch' / 'model', 'train')

In [10]:
train_set[0][0].shape

torch.Size([6, 128])

In [14]:
model_names = [ 'abmil']

models={}
for model_name in model_names:
    # print(model_name)
    model = get_model(model_name, input_dim, out_dim).to(device)
    trainer = TorchTrainer(model, criterion, merics, device=device, num_epochs=cfg['epoch'], batch_size=cfg['bs'])
    res = trainer.train(train_set, train_set)
    # res = trainer.train(fake, fake)
    models[model_name] = model


    pha = 'train'
    labels = np.array(res[pha][0])
    probs = np.array(res[pha][1])
    preds = np.array(res[pha][2])

    type_labels = labels[0]
    type_probs = probs[0]
    type_preds = preds[0]

    nonibc_index = np.where(type_labels ==1)
    nonibc_labels = labels[1][nonibc_index]
    nonibc_probs = probs[1][nonibc_index]
    nonibc_preds = preds[1][nonibc_index]

    ibc_index = np.where(type_labels ==2)
    ibc_labels = labels[1][ibc_index]
    ibc_probs = probs[1][ibc_index]
    ibc_preds = preds[1][ibc_index]

    re = {}
    avg_aucs = {}
    re['type'] = report(type_labels, type_preds, type_probs, class_names['type'])
    avg_aucs['type'] = compute_mean_one_vs_all_auc(type_labels, type_probs)

    re['nonibc'] = report(nonibc_labels, nonibc_preds, nonibc_probs, class_names['nonibc'])
    avg_aucs['nonibc'] = compute_mean_one_vs_all_auc(nonibc_labels, nonibc_probs)

    re['ibc'] = report(ibc_labels, ibc_preds, ibc_probs, class_names['ibc'])
    avg_aucs['ibc'] = compute_mean_one_vs_all_auc(ibc_labels, ibc_probs)

    for task in ['type', 'nonibc', 'ibc']:
        r = re[task]
        fs = "{} {} {} acc: {:.4f}, auc: {:.4f} [{:.4f} {:.4f} {:.4f}]".format(model_name, pha, task, r['accuracy'], avg_aucs[task], r['0']['auc'], r['1']['auc'], r['2']['auc'])
        print(fs)
    print()
print()
    

    


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [18:46<00:00,  5.63s/it]


abmil train type acc: 0.9818, auc: 0.9977 [0.9995 0.9956 0.9983]
abmil train nonibc acc: 0.9108, auc: 0.9695 [0.9744 0.9541 0.9810]
abmil train ibc acc: 0.9311, auc: 0.9809 [0.9899 0.9734 0.9794]




In [16]:
save_root = Path('/mnt/hd0/project/bcacad/model/wsi_models/model1')
if not save_root.exists():
    save_root.mkdir(parents=True)
for model_name in model_names:
    torch.save(models[model_name].state_dict(), save_root / f'{model_name}.pth')
    print(f'save {model_name}.pth')

save abmil.pth


In [11]:
model_names = [ 'abmil']
models={}
model_dir = Path('/mnt/hd0/project/bcacad/model/wsi_models/model1')
for model_name in model_names:
    model = get_model(model_name, input_dim, out_dim).to(device)
    model.load_state_dict(torch.load(model_dir / f'{model_name}.pth'))
    models[model_name] = model.to(device)

In [15]:
# testing
pha='test'
# test_cohorts = ['suqh_all_patch', 'qduh', 'shsu', 'bracs', 'bcnb', 'bach', 'apght']
test_cohorts = [ 'suqh_all_patch']
# load_dir = Path('/mnt/hd0/project/bcacad/model/roi_models/model2')
for cohort in test_cohorts:
    for model_name in model_names:
        # model = get_model(model_name, input_dim, out_dim).to(device)
        model = models[model_name]
        # ckpt = torch.load(load_dir / f'{model_name}.pth')
        # model.load_state_dict(ckpt)
        # model = model.to(device)

        test_set = WSI(feature_root / cohort / 'test', 'test')
        trainer = TorchTrainer(model, criterion, merics, device=device, num_epochs=50)
        res = trainer.predict(test_set)

        labels = np.array(res[0])
        probs = np.array(res[1])
        preds = np.array(res[2])

        type_labels = labels[0]
        type_probs = probs[0]
        type_preds = preds[0]

        nonibc_index = np.where(type_labels ==1)
        nonibc_labels = labels[1][nonibc_index]
        nonibc_probs = probs[1][nonibc_index]
        nonibc_preds = preds[1][nonibc_index]

        ibc_index = np.where(type_labels ==2)
        ibc_labels = labels[1][ibc_index]
        ibc_probs = probs[1][ibc_index]
        ibc_preds = preds[1][ibc_index]

        re = {}
        avg_aucs = {}
        if type_labels.size == 0:
            re['type'] = None
            avg_aucs['type'] = None
        else:
            re['type'] = report(type_labels, type_preds, type_probs, class_names['type'])
            avg_aucs['type'] = compute_mean_one_vs_all_auc(type_labels, type_probs)

        if nonibc_labels.size == 0:
            re['nonibc'] = None
            avg_aucs['nonibc'] = None
        else:
            re['nonibc'] = report(nonibc_labels, nonibc_preds, nonibc_probs, class_names['nonibc'])
            avg_aucs['nonibc'] = compute_mean_one_vs_all_auc(nonibc_labels, nonibc_probs)

        if ibc_labels.size == 0:
            re['ibc'] = None
            avg_aucs['ibc'] = None
        else:
            re['ibc'] = report(ibc_labels, ibc_preds, ibc_probs, class_names['ibc'])
            avg_aucs['ibc'] = compute_mean_one_vs_all_auc(ibc_labels, ibc_probs)

        for task in ['type', 'nonibc', 'ibc']:
            r = re[task]
            if r is None:
                continue
            fs = "{} {} {} acc: {:.4f}, auc: {:.4f} [{:.4f} {:.4f} {:.4f}]".format(cohort, model_name, task, r['accuracy'], avg_aucs[task], r['0']['auc'], r['1']['auc'], r['2']['auc'])
            print(fs)
        print()

suqh_all_patch abmil type acc: 0.9733, auc: 0.9927 [0.9982 0.9898 0.9891]
suqh_all_patch abmil nonibc acc: 0.5000, auc: 0.6800 [0.6514 0.6038 0.7903]
suqh_all_patch abmil ibc acc: 0.6842, auc: 0.7902 [0.8734 0.6859 0.8100]

